In [ ]:
# Painting vs. Photo Project - Classification with Convolutional Neural Network (Keras)
# Dataset source: https://www.kaggle.com/iiplutocrat45ii/painting-vs-photograph-classification-dataset

In [1]:
# Connect with Google Drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
# Load files from Google Drive
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/Paint_vs_Photo.zip", 'r')
zip_ref.extractall('/content/Paint_vs_Photo')
zip_ref.close()

In [9]:
# Directories

import os 

# Raw data / entire dataset
train_raw_paint_dir = os.path.join("/content/Paint_vs_Photo/RawData/painting")
train_raw_photo_dir = os.path.join("/content/Paint_vs_Photo/RawData/photos")

# Splitted into training / test / validation dataset
train_paint_dir = os.path.join("/content/Paint_vs_Photo/train/painting")
train_photo_dir = os.path.join("/content/Paint_vs_Photo/train/photos")

valid_paint_dir = os.path.join("/content/Paint_vs_Photo/valid/painting")
valid_photo_dir = os.path.join("/content/Paint_vs_Photo/valid/photos")

In [11]:
# Number of items in each dataset (directory)

print("Train_raw total - paintings:", len(os.listdir(train_raw_paint_dir)))
print("Train_raw total - photos:", len(os.listdir(train_raw_photo_dir)))
print("Train total - paintings:", len(os.listdir(train_paint_dir)))
print("Train total - photos:", len(os.listdir(train_photo_dir)))
print("Valid total - paintings:", len(os.listdir(valid_paint_dir)))
print("Valid total - photos:", len(os.listdir(valid_photo_dir)))

Train_raw total - paintings: 1361
Train_raw total - photos: 3747
Train total - paintings: 3513
Train total - photos: 3528
Valid total - paintings: 1505
Valid total - photos: 1505


In [24]:
# Display some pictures 
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

train_paint_names = os.listdir(train_paint_dir)

#img = 

#img=mpimg.imread()
#imgplot = plt.imshow(img)
#imgplot